In [1]:
from fastcore.all import *
from fastai.vision.widgets import *
from fastai.vision.all import *

import os
from os.path import join

import PIL
import numpy as np

import matplotlib.pyplot as plt

import torch.nn
import torch

from tqdm import tqdm

In [2]:
path = untar_data(URLs.MNIST)
path

Path('C:/Users/0rc44/.fastai/data/mnist_png')

In [3]:
train_path = path/'training'
test_path = path/'test'

In [4]:
labels = list()
data = list()
for root,dirs,files in os.walk(train_path):
    for file in files:
        if file.endswith(".png"):
            img = PIL.Image.open(join(root,file))
            data.append(np.asarray(img).flatten())
            labels.append(parent_label(join(root,file)))

# data[0]

labels = np.array(labels,dtype=int)
data = np.stack(data,axis=0)

In [5]:
data.shape

(60000, 784)

In [6]:
labels.shape

(60000,)

In [7]:
train_data = data / 255
test_data = labels.reshape(-1,1)

In [8]:
X = torch.tensor(train_data)
y = torch.tensor(test_data).long()

In [9]:
X

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)

In [10]:
y

tensor([[0],
        [0],
        [0],
        ...,
        [9],
        [9],
        [9]])

In [11]:
class Classifier(torch.nn.Module):
    def __init__(self,n_inputs,n_outputs):
        super(Classifier,self).__init__()
        self.seq = torch.nn.Sequential(
             torch.nn.Linear(in_features=n_inputs,out_features=128)
            ,torch.nn.ReLU()
            ,torch.nn.Linear(in_features=128,out_features=64)
            ,torch.nn.ReLU()
            ,torch.nn.Linear(in_features=64,out_features=n_outputs)
            ,torch.nn.Softmax(dim=1)
         
        )
        
    def forward(self,x):
        return self.seq(x.float())

In [12]:
data.shape

(60000, 784)

In [13]:
torch_model = Classifier(n_inputs = data.shape[-1],n_outputs = 10)

In [14]:
torch_model(X)

tensor([[0.1026, 0.0980, 0.1014,  ..., 0.1047, 0.1017, 0.1032],
        [0.1049, 0.1005, 0.0979,  ..., 0.1034, 0.1021, 0.1029],
        [0.1040, 0.0985, 0.0996,  ..., 0.1018, 0.1041, 0.1055],
        ...,
        [0.1082, 0.1005, 0.0989,  ..., 0.1038, 0.1047, 0.1009],
        [0.1080, 0.1000, 0.0976,  ..., 0.1013, 0.1021, 0.1030],
        [0.1046, 0.1028, 0.0967,  ..., 0.1034, 0.1042, 0.1020]],
       grad_fn=<SoftmaxBackward0>)

In [15]:
(torch_model(X).argmax(axis=1) == y.flatten()).float().mean()

tensor(0.0632)

In [20]:
epochs = 10000

criterion = torch.nn.NLLLoss()
optimizer = torch.optim.SGD(torch_model.parameters(),lr=0.01)


# training loop 
for epoch in tqdm(range(epochs)):
    y_pred = torch_model(X)
    loss = criterion(y_pred,y.flatten().long())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [37:21<00:00,  4.46it/s]


In [21]:
(torch_model(X).argmax(axis=1) == y.flatten()).float().mean()

tensor(0.6740)

In [22]:
torch_model(X)

tensor([[9.9998e-01, 1.4762e-12, 8.3507e-07,  ..., 2.6193e-07, 2.5947e-09,
         1.4121e-09],
        [9.9951e-01, 1.8488e-10, 9.7753e-05,  ..., 4.2005e-06, 6.5182e-07,
         9.8973e-07],
        [8.0722e-01, 8.2562e-09, 5.2497e-05,  ..., 2.4375e-04, 7.5642e-08,
         8.5401e-07],
        ...,
        [1.8946e-05, 1.1104e-07, 6.3709e-05,  ..., 1.2392e-02, 4.5193e-06,
         8.0505e-04],
        [1.1248e-04, 2.5736e-06, 6.7240e-06,  ..., 9.5774e-01, 9.7164e-06,
         4.8757e-04],
        [1.3517e-04, 9.1844e-05, 3.4854e-04,  ..., 5.0164e-02, 1.0709e-04,
         6.8043e-03]], grad_fn=<SoftmaxBackward0>)

In [23]:
print(torch_model)

Classifier(
  (seq): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
    (5): Softmax(dim=1)
  )
)
